In [1]:
# from transformers import BartConfig, BartTokenizer, BartModel

# config = BartConfig.from_pretrained("facebook/bart-large")
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
# model = BartModel.from_pretrained("facebook/bart-large")

In [2]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("dylanmengzhou/kobart-trans-en-ko-v2")
# model = AutoModelForSeq2SeqLM.from_pretrained("dylanmengzhou/kobart-trans-en-ko-v2")

In [7]:
# from transformers import pipeline

# pipe = pipeline("translation", model="circulus/kobart-trans-ko-en-v2")
# pipe("오늘 점심으로 스테이크를 먹었습니다.")

from transformers import pipeline

pipe = pipeline("translation", model="dylanmengzhou/kobart-trans-en-ko-v2")
print(pipe("The grace of our Lord Jesus Christ be with you all. Amen."))
print(pipe("Therefore shall her plagues come in one day, death, and mourning, and famine; and she shall be utterly burned with fire: for strong is the Lord God who judgeth her."))
print(pipe("And the kings of the earth, who have committed fornication and lived deliciously with her, shall bewail her, and lament for her, when they shall see the smoke of her burning,"))


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Your input_length: 31 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Your input_length: 91 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': '우리 주 예수 그리스도의 은혜가 너희와 함께 하십시요. 아멘. 주'}]


Your input_length: 90 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': '그러므로 하루 아침에 역병이 몰려와 죽고 애통함과 기근이 오고 불로'}]
[{'translation_text': '그리고 그녀와 함께 살면서 처절한 시련을 겪고 맛있게 살았던 땅의'}]


In [4]:

import pymysql

# 데이터 베이스 연결하기
conn = pymysql.connect(host='1.251.203.204',
                       user='root',
                       password='kdt5',
                       db='Team4',
                       charset='utf8',
                       port=33065)

curs = conn.cursor()

# 검색 명령어 사용 
sql = "SELECT eng.text as eng_text, kor.text as kor_text FROM language_en eng join language_ko kor on eng.id = kor.id;"
curs.execute(sql)
result = curs.fetchall()
print("현재 테이블의 데이터수는 총 {}개 입니다.".format(len(result)))

현재 테이블의 데이터수는 총 31102개 입니다.


In [5]:
import pandas as pd
df = pd.read_sql(sql, conn)

# 데이터베이스 연결 종료
conn.close()

# 쿼리 -> dataframe결과 출력
df.head()

C:\Users\kdp\AppData\Local\Temp\ipykernel_24900\1190723113.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,eng_text,kor_text
0,In the beginning God created the heaven and th...,태초에 하나님이 천지를 창조하시니라
1,"And the earth was without form, and void; and ...",땅이 혼돈하고 공허하며 흑암이 깊음 위에 있고 하나님의 신은 수면에 운행하시니라
2,"And God said, Let there be light: and there wa...",하나님이 가라사대 빛이 있으라 하시매 빛이 있었고
3,"And God saw the light, that it was good: and G...",그 빛이 하나님의 보시기에 좋았더라 하나님이 빛과 어두움을 나누사
4,"And God called the light Day, and the darkness...",빛을 낮이라 칭하시고 어두움을 밤이라 칭하시니라 저녁이 되며 아침이 되니 이는 첫째...


In [6]:
def dataframe_iterator(df):
    for i, row in df.iterrows():
        yield row["eng_text"], row["kor_text"] 

split_idx = int(len(df) * 0.8)  # 전체 데이터 중 80%를 훈련 데이터셋으로 사용
train_df = df.iloc[:] # 걍 다 쓰세욤
valid_df = df.iloc[split_idx:]

In [7]:
train_dict = {"id":[],
              "translation":[]}
cnt = 0
for value in train_df.values:
    
    train_dict["id"].append(cnt)
    
    tempdict = {}
    tempdict["eng"] = value[0]
    tempdict["kor"] = value[1]
    train_dict["translation"].append(tempdict)
    # print(value)
    cnt += 1
    # break

train_df = pd.DataFrame(train_dict)

In [8]:
val_dict = {"id":[],
              "translation":[]}
cnt = 0
for value in valid_df.values:
    
    val_dict["id"].append(cnt)
    
    tempdict = {}
    tempdict["eng"] = value[0]
    tempdict["kor"] = value[1]
    val_dict["translation"].append(tempdict)
    # print(value)
    cnt += 1
    # break

valid_df = pd.DataFrame(val_dict)

In [9]:
from datasets import DatasetDict, Dataset

split_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(valid_df)  # 테스트 데이터도 동일한 데이터프레임을 사용하므로 train과 동일하게 처리
})

split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 31102
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 6221
    })
})

In [10]:
split_datasets["train"][1]["translation"]

{'eng': 'And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.',
 'kor': '땅이 혼돈하고 공허하며 흑암이 깊음 위에 있고 하나님의 신은 수면에 운행하시니라'}

In [11]:
from transformers import AutoTokenizer

model_checkpoint = "dylanmengzhou/kobart-trans-en-ko-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [12]:
en_sentence = split_datasets["train"][1]["translation"]["eng"]
ko_sentence = split_datasets["train"][1]["translation"]["kor"]

inputs = tokenizer(en_sentence, text_target=ko_sentence)
inputs

{'input_ids': [14603, 309, 299, 21235, 20041, 15562, 16160, 20676, 19542, 20676, 304, 16160, 17884, 315, 21579, 29709, 24536, 20290, 310, 18509, 258, 24536, 18785, 15562, 306, 309, 22873, 20676, 19542, 1700, 316, 311, 14889, 21235, 17254, 26605, 19524, 21235, 18785, 300, 29686, 245, 14603, 309, 299, 21235, 14470, 311, 21738, 16805, 19524, 15464, 20223, 19281, 310, 317, 17752, 1700, 316, 311, 14889, 21235, 17254, 26605, 19524, 21235, 20676, 16881, 27621, 245], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [15495, 12034, 14929, 9869, 14058, 14061, 13644, 14363, 16816, 117

In [13]:
wrong_targets = tokenizer(ko_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"])) 
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁땅', '이', '▁혼', '돈', '하고', '▁공', '허', '하며', '▁흑', '암', '이', '▁깊', '음', '▁위에', '▁있고', '▁하나님의', '▁신', '은', '▁수', '면에', '▁운행', '하시', '니라']
['▁땅', '이', '▁혼', '돈', '하고', '▁공', '허', '하며', '▁흑', '암', '이', '▁깊', '음', '▁위에', '▁있고', '▁하나님의', '▁신', '은', '▁수', '면에', '▁운행', '하시', '니라']


In [14]:
max_length = 128 

def preprocessing_function(examples):
    inputs = [ex["eng"] for ex in examples["translation"]]
    targets = [ex["kor"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

In [15]:
tokenized_datasets = split_datasets.map(
	preprocessing_function, 
    batched=True,
    remove_columns=split_datasets["train"].column_names)

Map:   0%|          | 0/31102 [00:00<?, ? examples/s]

Map: 100%|██████████| 6221/6221 [00:00<00:00, 11416.70 examples/s]


In [16]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [19]:
batch["labels"]

tensor([[15495, 12034, 14929,  9869, 14058, 14061, 13644, 14363, 16816, 11711,
         12034, 15397, 12013, 16782, 15438, 21420, 14094, 12005, 14032, 18466,
         19808, 18940, 26418],
        [14392, 16613, 22279, 11207,  9776, 27888, 17698, 10213, 20853, 10518,
         27888, 22658,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100]])

In [20]:
batch["decoder_input_ids"]

tensor([[    1, 15495, 12034, 14929,  9869, 14058, 14061, 13644, 14363, 16816,
         11711, 12034, 15397, 12013, 16782, 15438, 21420, 14094, 12005, 14032,
         18466, 19808, 18940],
        [    1, 14392, 16613, 22279, 11207,  9776, 27888, 17698, 10213, 20853,
         10518, 27888, 22658,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3]])

In [21]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[15495, 12034, 14929, 9869, 14058, 14061, 13644, 14363, 16816, 11711, 12034, 15397, 12013, 16782, 15438, 21420, 14094, 12005, 14032, 18466, 19808, 18940, 26418]
[14392, 16613, 22279, 11207, 9776, 27888, 17698, 10213, 20853, 10518, 27888, 22658]


# Metrics

In [22]:
from datasets import load_metric

metric = load_metric("sacrebleu")

C:\Users\kdp\AppData\Local\Temp\ipykernel_24900\200089639.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [23]:
predictions = [
    "In the beginning God created the heaven and the earth."
]
references = [
    [
    "태초에 하나님이 천지를 창조하시니라"
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [0, 0, 0, 0],
 'totals': [11, 10, 9, 8],
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 11,
 'ref_len': 4}

In [24]:
predictions = [
    "And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so."
]
references = [
    [
    "하나님이 궁창을 만드사 궁창 아래의 물과 궁창 위의 물로 나뉘게 하시매 그대로 되니라"
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [0, 0, 0, 0],
 'totals': [29, 28, 27, 26],
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 29,
 'ref_len': 13}

In [25]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    # 모델이 예측 logits 이상을 반환하는 경우
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # 라벨의 -100을 대체하여 디코딩할 수 없는 경우를 처리합니다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # 간단한 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [27]:
from transformers import Seq2SeqTrainingArguments 

args = Seq2SeqTrainingArguments(
	f"dylanmengzhou/kobart-trans-en-ko-v2",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
)

In [ ]:
# import transformers

# transformers.__version__

'4.40.0'

In [30]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6624eda3-65eeec410f1b774f50db9990;81eb6416-5c32-446e-bf8e-a4216047712e)

Invalid username or password.